### Assignment 1:
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
# Set environment variables for OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

# Set environment variables for LangChain
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Set environment variables for Groq
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "")

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate


class ProductInfo(BaseModel):
    product_name: str = Field(description="Name of the product")
    product_details: str = Field(description="Details of the product")
    price_usd: int = Field(description="Price of the product in USD")

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    max_retries=3,
    request_timeout=60,
)

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=ProductInfo)

# prompt = PromptTemplate(
#     template="Answer the user query.\n{format_instructions}\n{query}\n",
#     input_variables=["query"],
#     partial_variables={"format_instructions": parser.get_format_instructions()},
# )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a product expert assistant. Only return real-world products that actually exist. Don't make up product names."),
        (
            "user",
            "Answer the user query.\n{format_instructions}\n{query}\n",
        ),
    ]
).partial(format_instructions=parser.get_format_instructions())

chain = prompt | llm | parser
response = chain.invoke(
    {
        "query": "What is the name, details, and price of the product that is a smartphone with a 6.8 inch display and Hasselblad Camera?"
    }
)

print(response)

{'product_name': 'OnePlus 11 5G', 'product_details': 'The OnePlus 11 5G features a 6.8-inch AMOLED display and a Hasselblad Camera for Mobile, offering advanced photography capabilities.', 'price_usd': 699}
